In [48]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
from scipy.optimize import curve_fit

In [27]:
exps = ['GramA 2.6nM','GramA 260pM20210920','GUV','OmpF']
path_identifier = ['3nM','260pM20210920','GUV','OmpF']

paths = np.array(glob('Permeabilities*.csv'))
print(paths)
exp_hash = {}
positions = np.arange(len(paths))

for i,path_id in enumerate(path_identifier):
    for position,path in enumerate(paths[positions]):
        if path.find(path_id) != -1:
            
            exp_hash[exps[i]] = path
#             positions = np.concatenate((positions[:position],positions[position+1:]))
            print(positions)

['Permeabilities_GramicidinA_260pM20210920.csv'
 'Permeabilities_GramicidinA_3nM_20210920.csv'
 'Permeabilities_OmpF_20210427.csv' 'Permeabilities_GUV.csv']
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]
[0 1 2 3]


In [28]:
exp_hash

{'GramA 2.6nM': 'Permeabilities_GramicidinA_3nM_20210920.csv',
 'GramA 260pM20210920': 'Permeabilities_GramicidinA_260pM20210920.csv',
 'GUV': 'Permeabilities_GUV.csv',
 'OmpF': 'Permeabilities_OmpF_20210427.csv'}

['Permeabilities_GramicidinA_260pM20210920.csv',
 'Permeabilities_GramicidinA_3nM_20210920.csv',
 'Permeabilities_OmpF_20210427.csv',
 'Permeabilities_GUV.csv']

In [29]:

dfs = {exps[i]: pd.read_csv(exp_hash[exps[i]]) for i in range(len(exps))}

In [31]:
%matplotlib qt
fig, axs  = plt.subplots(1,len(dfs.keys()))

for i,key in enumerate(dfs.keys()):
    
    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    P = dfs[key]['P cms^-1'].to_numpy().astype(float)
    rel_err = rel_err[P > 0]
    P = P[P > 0]

    
    axs[i].hist(np.log10(P[rel_err < 0.1]),'sturges')
    
    x_lab = [-8,-6.5,-5]
    y_lab = [0,5,10]
    
    axs[i].set_xticks(x_lab)
    axs[i].set_xticklabels(x_lab,fontsize = 15)
    
    axs[i].set_yticks(y_lab)
    axs[i].set_yticklabels(y_lab,fontsize = 15)
    
    axs[i].set_xlim([-8.5,-4.5])
    axs[i].set_ylim([0,11])
    
    axs[i].set_title(key)

In [32]:
print(plt.rcParams['axes.prop_cycle'].by_key()['color'])

['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']


In [35]:
# now when happy with the settings arrange on one plot
fig, ax  = plt.subplots(1,1)

for i,key in enumerate(dfs.keys()):
    
    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    P = dfs[key]['P cms^-1'].to_numpy().astype(float)
    rel_err = rel_err[P > 0]
    P = P[P > 0]

    color = plt.rcParams['axes.prop_cycle'].by_key()['color'][i]
    
    ax.hist(np.log10(P[rel_err < 0.1]),'sturges',label = key,alpha = 0.3,
            edgecolor = color)
    
x_lab = [-8,-6.5,-5]
y_lab = [0,5,10]

ax.set_xticks(x_lab)
ax.set_xticklabels(x_lab,fontsize = 15)

ax.set_yticks(y_lab)
ax.set_yticklabels(y_lab,fontsize = 15)

ax.set_xlim([-8.5,-4.5])
ax.set_ylim([0,11])

# ax.set_title(key)

ax.set_xlabel('$Log_{10} (P [cms^{-1}])$',fontsize = 18)
ax.set_ylabel('# GUVs',fontsize = 18)
# plt.legend()

plt.tick_params(direction = 'in',top= True, right = True,left = True, bottom = True, length = 6)


# Fit conductance vs concentration for OmpF


In [47]:
dfGvsK = pd.DataFrame({'[K][molm^-3]': [100,500,1000,1500],'G [S]':[0.5e-9,2e-9,4.5e-9,5.25e-9]})

In [50]:
# fitting function

def GfromC(C,P):
    F = 96485
    R = 8.31
    T = 298
    
    return (F**2*C*P)/(R*T)

guess = [2e-18]

# fit using curve fitabs
params = curve_fit(GfromC,dfGvsK['[K][molm^-3]'],dfGvsK['G [S]'],p0 = guess)

(array([1.01743076e-18]), array([[4.80305164e-39]]))

In [36]:
#{ key: [Conductance,Potassium concentration when measured]}

conductances = {'OmpF':[8e-10,100],'GramA':[1.4e-11,100]}


def PfromG(G,C,R = 8.31,T = 298,F = 96485):
    #make sure C is given as molm^-3 for units to make sense
    #also G given as Siemens (Amperes/Volts)
    
    #returns P in m^3/s
    return G*(R*T)/(F**2*C)
   
    
    
exp_one_P = {key: PfromG(*conductances[key]) for key in conductances.keys()}

In [52]:
# use fit for permeability of one OmpF pore

exp_one_P['OmpF'] = params[0]

In [53]:
App = 3.16 #um^2
for key in exps:
    
    if key == 'GUV':
        continue
        
    if key.find('GramA') != -1:
        id_ = 'GramA'
        
    else:
        id_  = key
        
    dfs[key]['P1_exp ms^-1'] = exp_one_P[id_]/(dfs[key]['A']*App*1e-12)
    dfs[key]['N'] = dfs[key]['P ms^-1']/dfs[key]['P1_exp ms^-1']

In [63]:
# process Number of pores per GUV to make sense
key = 'OmpF'


dfs[key]['N'].iloc[dfs[key]['N'].to_numpy().astype(float) < 0] = 0

/home/marcus/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [65]:
dfs['OmpF']

,Unnamed: 0,P ms^-1,dP/P,A,P cms^-1,P1_exp ms^-1,N
0,0,1.350808e-08,0.071106,133.000000,1.350808e-06,2.420840e-09,5.579916
1,1,9.869692e-09,0.047763,220.100000,9.869692e-07,1.462843e-09,6.746925
2,2,-2.220650e-08,-0.050385,89.633333,-2.220650e-06,3.592098e-09,0.000000
3,3,7.582919e-09,0.175472,251.266667,7.582919e-07,1.281395e-09,5.917708
4,4,9.295701e-09,0.028209,220.033333,9.295701e-07,1.463286e-09,6.352619
5,5,7.427063e-09,0.145010,189.433333,7.427063e-07,1.699657e-09,4.369741
6,6,1.298657e-08,0.070342,155.666667,1.298657e-06,2.068341e-09,6.278739
7,7,1.136544e-08,0.069337,154.266667,1.136544e-06,2.087112e-09,5.445534
8,8,6.166686e-09,0.066939,135.966667,6.166686e-07,2.368020e-09,2.604153
9,9,4.194933e-08,0.037295,192.500000,4.194933e-06,1.672581e-09,25.080603


In [64]:
dfs[key]['N']



0      5.579916
1      6.746925
2      0.000000
3      5.917708
4      6.352619
5      4.369741
6      6.278739
7      5.445534
8      2.604153
9     25.080603
10    19.133293
11    26.601090
12     3.630193
13     4.436284
14     5.523107
15     3.092623
16     9.480269
17     3.083036
18     4.293859
19     6.981089
20    10.678770
21    14.043688
22    13.812505
23     9.182059
24     1.615958
25     9.500955
26     1.985614
27    18.573839
Name: N, dtype: float64

In [54]:
for key in dfs.keys():
    if key == 'GUV':
        
        continue
        
    print(key,dfs[key]['N'])

GramA 2.6nM 0     6845.569760
1     2356.839317
2     3487.236081
3      952.853246
4     3153.913718
5      349.914106
6      526.579639
7      445.411879
8      778.507495
9       23.949132
10     258.987791
11    2228.040901
12    5608.477045
13    3287.102811
14    4921.210064
15     596.328812
16    1833.403215
Name: N, dtype: float64
GramA 260pM20210920 0        25.230959
1       136.875373
2        70.585646
3       165.361716
4       167.398634
5       366.149309
6        86.896769
7        36.171886
8       182.091422
9       388.881234
10      157.243911
11       61.319097
12      508.552980
13       92.784491
14      324.911753
15      209.396575
16      127.532977
17      629.871351
18      375.727153
19     1338.825028
20     2989.685748
21      778.196259
22      715.349455
23     1168.472546
24        5.906141
25      370.257503
26     2075.100605
27        1.941277
28      524.048900
29      230.330685
          ...     
73      601.181989
74       61.314296
75       62

In [55]:
%matplotlib qt
fig, axs  = plt.subplots(1,len(dfs.keys()))

for i,key in enumerate(dfs.keys()):
    
    if key == 'GUV':
        continue
    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    N = dfs[key]['N'].to_numpy().astype(float)
    rel_err = rel_err[N > 0]
    N = N[N > 0]

    
    axs[i].hist(np.log10(N[rel_err < 0.1]),'sqrt')
    

In [45]:
# Gramicidin A plot

keys = ['GramA 2.6nM','GramA 260pM20210920']

fig, ax = plt.subplots(1,1)


for i,key in enumerate(keys):

    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    N = dfs[key]['N'].to_numpy().astype(float)
    
    color = plt.rcParams['axes.prop_cycle'].by_key()['color'][i]
    ax.hist(np.log10(N[rel_err < 0.1]),5,label = key,alpha = 0.3,
            edgecolor = color)
    

x_ticks = [2,3,4]
y_ticks= [0,4,8]

  
plt.tick_params(direction = 'in',top= True, right = True,left = True, bottom = True, length = 6)

ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticks,fontsize = 15)
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks,fontsize = 15)

ax.set_ylabel('# GUVs',fontsize = 18)
ax.set_xlabel('$Log_{10}$(No. of pores per GUV)', fontsize = 18)

ax.set_xlim([1.9,4.1])
plt.legend()


In [76]:
# OmpF plot
keys = ['OmpF']

fig, ax = plt.subplots(1,1)


for i,key in enumerate(keys):

    rel_err = dfs[key]['dP/P'].to_numpy().astype(float)
    N = dfs[key]['N'].to_numpy().astype(float)
    
    color = plt.rcParams['axes.prop_cycle'].by_key()['color'][i]
    ax.hist(N[rel_err < 0.1],5,label = key,alpha = 0.3,
            edgecolor = color)
    

x_ticks = [0,8,16,24]
y_ticks= [0,4,8]

  
plt.tick_params(direction = 'in',top= True, right = True,left = True, bottom = True, length = 6)

ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticks,fontsize = 15)
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks,fontsize = 15)

ax.set_ylabel('# GUVs',fontsize = 18)
ax.set_xlabel('$No. of pores per GUV', fontsize = 18)

# ax.set_xlim([1.9,4.1])
plt.legend()
